<div style='width:400px; height:250px; border-bottom:10px black solid;border-left: 10px green solid;padding: 10px 10px 10px 10px;'>
    <h1> ---Timeseries forcasting of the Norwegian salmon exportation---</h1>
    <h3> Author: Jakob Lindstrøm, Joakim Sælemyr, Henrik Knudsen, Marcus Hjertaas </h3>
    <h3> Date: 29.05.2023 </h3>
</div>

<div style='width:600px; height:950px; padding: 10px 10px 10px 10px;border-bottom:10px black solid;
            border-left: 10px green solid'>
    <div>
        <h4>Keywords:</h4>
        <p> Salmon, timeseries, SARIMAX </p>
        <h4> Data sources:  </h4>
        <p> MET: https://api.met.no/ </p>
        <p> geonorge: https://www.geonorge.no/ </p>
        <h4> Intention: </h4>
        <p> This programe is a modified version of the coding part for me and my fellow students when we wrote our bachelor thesis. This modified version separates itself from what we delivered in the matter that I removed a lot of the exploratory dataframe analysis, I have merge all our notebooks into the same notebook, I have cleaned some code and added a fifth classification model: Naive Bayes. </p>
        <div>
            <h2>Executive summary </h2>
            <p>
            The purpose of this bachelor thesis is to find the best machine learning method to accurately 
            predict bank term deposits from the public. We are viewing the issue from a business 
            perspective, also taking real-world implications into account.  
                <br><br>
            The data which the models are applied to provides a series of different information from a 
            sample of Portuguese individuals, including whether or not these individuals have made 
            committed long-term deposits into the bank. The data have gone through a set of 
            configurations and resampling techniques. Retrieval of the data was in the wake of the Great 
            Financial Crisis of 2008. 
                <br><br>            
            Four machine learning models were chosen for prediction, they are all based on 
            acknowledged statistical classification methods, these are: binomial logistic regression, 
            decision tree classifier, artificial neural networks, and support vector machine. These methods 
            vary in complexity and have different sets of advantages and disadvantages. They will be 
            applied to face the prediction-challenges in the data. It’s not possible to combine one method 
            to cover another method's disadvantages. However, by using several methods it is believed 
            that one can find the model that best faces the specific challenges provided by the data. 
            The best model is based on a comprehensive assessment, including two criterions. The first 
            criterion is based on the prediction rate and the model’s ability at predicting actual possible 
            customers. The second criterion concerns the model's training data and its amount of 
            resampling interference. 
                <br><br>
                The models are configured to give predictions for the conditions during the period of dataretrieval and are therefore constrained to these conditions. According to our findings, the 
            support vector machine model trained on the undersampled data is the most favorable model. 
            The model showed both great prediction accuracy and managed to classify the minority class 
            precisely.
            </p>
        </div>   
    </div>
</div>

<h2> Importations of libraries

In [49]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score as cvs
from sklearn.preprocessing import scale

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from seaborn import heatmap as hm

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1, l2

from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier as dtc
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings('ignore')

<h2> Resampling </h2>
<p> Splitting the dataframe into a test set and three train sets. The three train sets are based on different resampling techniques

In [2]:
df = pd.read_csv('original.csv')
del df['Unnamed: 0']

#Splitting into original train set and universal test size
orgtrain, test = train_test_split(df, test_size=.2, random_state=42)

#Retriving all 1s from original train set
ones = orgtrain.loc[orgtrain.Deposit == 1]

#Retriving random zeros from the original train set
alfa, randomzeros = train_test_split(orgtrain.loc[orgtrain.Deposit==0], test_size=.13255, random_state=42)

under = pd.concat([ones, randomzeros], axis=0)

#Retrieving all 0s from original train set
allzeros = orgtrain.loc[orgtrain.Deposit == 0]

#Replicating all the ones from original train set seven times,
#and concats it into one df
newone = ones
for i in range(0,6,1):
    newone = pd.concat([newone,ones],axis=0)

#Retrieving random 1s from original train set 
#so the amount of 1s and 0s is matching.
charlie, additionalones = train_test_split(ones, test_size=0.54429, random_state=42)

#adding all the ones to create one large df with only 1s.
totalones = pd.concat([newone,additionalones], axis=0)

#Concating the 0s and 1s into one oversample df.
over = pd.concat([totalones, allzeros],axis=0)

In [3]:
#Downloading csv-files
# test = pd.read_csv('test.csv')
# orgtrain = pd.read_csv('org.csv')
# over  = pd.read_csv('over.csv')
# under = pd.read_csv('under.csv')
# del test['Unnamed: 0.1']
# del test['Unnamed: 0']
# del orgtrain['Unnamed: 0']
# del over['Unnamed: 0']
# del under['Unnamed: 0']

<h2> Specifying targets and data to be used in the models

In [4]:
column  = test.columns.values.tolist()
column.remove('Deposit')

In [5]:
testtarget = test[['Deposit']]
testdata = test[column]

org_train_target= orgtrain[['Deposit']]
org_train_data= orgtrain[column]

under_train_target = under[['Deposit']]
under_train_data = under[column]

over_train_target = over[['Deposit']]
over_train_data= over[column]

datas = [org_train_data,under_train_data,over_train_data,testdata]
targets = [org_train_target,under_train_target,over_train_target,testtarget]
idx = ['Original', 'Under','Over']

<h2> Logistic Regression

In [6]:
def LogReg():
    model = LogisticRegression(solver='lbfgs',max_iter=95000 )         
    

    train_res = []
    test_res = []
    pcl_res = []

    for i in range(0,3,1):
        model = model.fit(datas[i],targets[i])
        train_res_sel = model.score(datas[i],targets[i])
        test_res_sel = model.score(testdata,testtarget)
        cm = confusion_matrix(model.predict(testdata),testtarget) 
        cm = cm/np.sum(cm)
        cm = cm[0][1]/(cm[0][1]+cm[0][0]) 
               
        train_res.append(train_res_sel)
        test_res.append(test_res_sel)
        pcl_res.append(cm)

    df = pd.DataFrame({'Train':train_res,'Test':test_res,'PCL':pcl_res})

    return df

C:\Users\Jakob\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jakob\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jakob\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


<h2> Naive Bayes 

In [34]:
def NaiBay():
    model = GaussianNB()         

    train_res = []
    test_res = []
    pcl_res = []

    for i in range(0,3,1):
        model = model.fit(datas[i],targets[i])
        train_res_sel = model.score(datas[i],targets[i])
        test_res_sel = model.score(testdata,testtarget)
        cm = confusion_matrix(model.predict(testdata),testtarget)
        cm = cm/np.sum(cm)
        cm = cm[0][1]/(cm[0][1]+cm[0][0]) 
               
        train_res.append(train_res_sel)
        test_res.append(test_res_sel)
        pcl_res.append(cm)
                
#         cm = confusion_matrix(model.predict(datas[i]), targets[i])
#         cm = cm/np.sum(cm)
#         plt.figure(figsize=(6,6))
#         sns.heatmap(cm, annot=True,fmt='0.3%', cmap='Blues')
#         plt.ylabel("Actual")
#         plt.xlabel("Predicted")

    df = pd.DataFrame({'Train':train_res,'Test':test_res,'PCL':pcl_res})

    return df

<h2> Artifical Neural Networks

In [47]:
def ANN():
    for i in range(0,3,1):
        # Scaling the numeric variables
        num_vars = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

        for i in range(0,4,1):
            scaler = MinMaxScaler()
            datas[i][num_vars] = scaler.fit_transform(datas[i][num_vars])


    train_res = []
    test_res = []
    pcl_res = []

    for i in range(0,3,1):
        model = Sequential()
        model.add(Dense(32, input_dim=datas[i].shape[1], activation="leaky_relu"))
        model.add(Dense(16, activation="leaky_relu"))
        model.add(Dense(1, activation="sigmoid"))

        # Compile the model
        model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
        # Train the model
        model.fit(x=datas[i], y=targets[i], epochs=20, batch_size=10, verbose=1, validation_split=.1)

        # Train 
        predictions = model.predict(x=datas[i], batch_size=10, verbose=0)
        rounded_pred = [np.round(num) for num in predictions]
        cm = confusion_matrix(rounded_pred,targets[i])
        cm = cm/np.sum(cm)
        train_res_sel = cm[0][0] + cm[1][1]

        # Test and CLP
        predictions_test = model.predict(x=testdata, batch_size=10, verbose=0)
        rounded_pred_test = [np.round(num) for num in predictions_test]
        cm = confusion_matrix(rounded_pred_test,testtarget)
        cm = cm/np.sum(cm)
        test_res_sel = cm[0][0] +  cm[1][1]
        cm = cm[0][1]/(cm[0][1]+cm[0][0]) 

        train_res.append(train_res_sel)
        test_res.append(test_res_sel)
        pcl_res.append(cm)

    df = pd.DataFrame({'Train':train_res,'Test':test_res,'PCL':pcl_res})

    return df

<h2> Decision Tree Classifier

In [9]:
def DTC():
    
    train_res = []
    test_res = []
    pcl_res = []
    
    for i in range(0,3,1):
        model = dtc().fit(datas[i], targets[i])

        path = model.cost_complexity_pruning_path(datas[i], targets[i])
        ccp_alphas = path.ccp_alphas
        ccp_alphas = ccp_alphas[:-1]
        
        model = dtc()
        scores = cvs(model, datas[i], targets[i], cv=2)

        alpha_loop_values = []
        for alpha in ccp_alphas:
            model = dtc(ccp_alpha=alpha, random_state=42)
            scores = cvs(model, datas[i], targets[i], cv=5)
            alpha_loop_values.append([alpha, np.mean(scores), np.std(scores)])

        alpha_results = pd.DataFrame(alpha_loop_values, columns = ["Alpha", "Mean_Accuracy", "STD_Accuracy"])

        # Selecting the optimal alpha, based on maximum mean accuracy
        ideal_alpha = alpha_results[alpha_results.Mean_Accuracy == max(alpha_results.Mean_Accuracy)].Alpha
        if len(ideal_alpha) > 1:
            ideal_alpha = ideal_alpha.iloc[-1]
        ideal_alpha = float(ideal_alpha)

        model = dtc(ccp_alpha = ideal_alpha)
        model = model.fit(datas[i], targets[i])

        train_res_sel = model.score(datas[i],targets[i])
        test_res_sel = model.score(testdata,testtarget)
        cm = confusion_matrix(model.predict(testdata),testtarget)
        cm = cm/np.sum(cm)
        cm = cm[0][1]/(cm[0][1]+cm[0][0]) 
               
        train_res.append(train_res_sel)
        test_res.append(test_res_sel)
        pcl_res.append(cm)

    df = pd.DataFrame({'Train':train_res,'Test':test_res,'PCL':pcl_res})

    return df

<h2> Support Vector Machine

In [10]:
def SVM():

    train_res = []
    test_res = []
    pcl_res = []

    for i in range(0,3,1):

        datas_scaled = scale(datas[i])
        targets_scaled = scale(targets[i])

        model = SVC(random_state=42, C=5, gamma=0.01)
        model = model.fit(datas_scaled, targets[i])

        train_res_sel = model.score(datas_scaled,targets[i])
        test_res_sel = model.score(scale(datas[3]),testtarget)
        cm = confusion_matrix(model.predict(scale(testdata)),testtarget)
        cm = cm/np.sum(cm)
        cm = cm[0][1]/(cm[0][1]+cm[0][0]) 
               
        train_res.append(train_res_sel)
        test_res.append(test_res_sel)
        pcl_res.append(cm)

    df = pd.DataFrame({'Train':train_res,'Test':test_res,'PCL':pcl_res})

    return df

<h2> Retrieving the results 

In [56]:
LogRegres = LogReg()
LogRegres = LogRegres.set_index(pd.Series(idx))
LogRegres

,Train,Test,PCL
Original,0.902096,0.894946,0.062812
Under,0.832859,0.813668,0.022696
Over,0.832702,0.759372,0.014948


In [55]:
NaiBayres = NaiBay()
NaiBayres = NaiBayres.set_index(pd.Series(idx))
NaiBayres

,Train,Test,PCL
Original,0.862282,0.835674,0.053453
Under,0.702584,0.842420,0.061416
Over,0.711664,0.811235,0.046741


In [50]:
ANNres = ANN()
ANNres = ANNres.set_index(pd.Series(idx))
ANNres

Epoch 1/20
3256/3256 [==============================] - 4s 1ms/step - loss: 0.2714 - accuracy: 0.8946 - val_loss: 0.2298 - val_accuracy: 0.9057
Epoch 2/20
3256/3256 [==============================] - 4s 1ms/step - loss: 0.2285 - accuracy: 0.9029 - val_loss: 0.2193 - val_accuracy: 0.9066
Epoch 3/20
3256/3256 [==============================] - 4s 1ms/step - loss: 0.2210 - accuracy: 0.9034 - val_loss: 0.2118 - val_accuracy: 0.9082
Epoch 4/20
3256/3256 [==============================] - 4s 1ms/step - loss: 0.2164 - accuracy: 0.9045 - val_loss: 0.2137 - val_accuracy: 0.9068
Epoch 5/20
3256/3256 [==============================] - 4s 1ms/step - loss: 0.2131 - accuracy: 0.9061 - val_loss: 0.2070 - val_accuracy: 0.9135
Epoch 6/20
3256/3256 [==============================] - 4s 1ms/step - loss: 0.2092 - accuracy: 0.9059 - val_loss: 0.2186 - val_accuracy: 0.9082
Epoch 7/20
3256/3256 [==============================] - 4s 1ms/step - loss: 0.2069 - accuracy: 0.9065 - val_loss: 0.2029 - val_accuracy:

Epoch 18/20
5728/5728 [==============================] - 7s 1ms/step - loss: 0.2725 - accuracy: 0.8935 - val_loss: 0.4169 - val_accuracy: 0.8061
Epoch 19/20
5728/5728 [==============================] - 6s 1ms/step - loss: 0.2704 - accuracy: 0.8931 - val_loss: 0.4510 - val_accuracy: 0.7904
Epoch 20/20
5728/5728 [==============================] - 6s 1ms/step - loss: 0.2691 - accuracy: 0.8950 - val_loss: 0.3474 - val_accuracy: 0.8368


,Train,Test,PCL
Original,0.915035,0.881013,0.037281
Under,0.876956,0.771868,0.012903
Over,0.891029,0.732500,0.010441


In [45]:
DTCres = DTC()
DTCres = DTCres.set_index(pd.Series(idx))
DTCres

,Train,Test,PCL
Original,0.905607,0.867522,0.048409
Under,0.870673,0.767113,0.020820
Over,0.999890,0.844299,0.091511


In [42]:
SVMres =  SVM()
SVMres = SVMres.set_index(pd.Series(idx))
SVMres

C:\Users\Jakob\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jakob\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jakob\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,Train,Test,PCL
Original,0.921063,0.902687,0.082132
Under,0.888454,0.696340,0.009238
Over,0.900787,0.732279,0.022778


<h2> Combining the results into one dataframe

In [146]:
results = [LogRegres, NaiBayres,ANNres, DTCres, SVMres]
methods = ['Logistic','Naive  Bayes','ANN','DTC','SVM']
datasets = ['Original', 'Under','Over']

for i in range(0,len(results),1):
    labels = [np.repeat(methods[i],3),datasets]
    tuples = list(zip(*labels))
    index = pd.MultiIndex.from_tuples(tuples, names=['Method', 'Dataset'])
    results[i] = results[i].set_index(index)
resultsdf = pd.concat(results)

In [147]:
resultsdf

Train      Test       PCL
Method       Dataset                               
Logistic     Original  0.902096  0.894946  0.062812
             Under     0.832859  0.813668  0.022696
             Over      0.832702  0.759372  0.014948
Naive  Bayes Original  0.862282  0.835674  0.053453
             Under     0.702584  0.842420  0.061416
             Over      0.711664  0.811235  0.046741
ANN          Original  0.915035  0.881013  0.037281
             Under     0.876956  0.771868  0.012903
             Over      0.891029  0.732500  0.010441
DTC          Original  0.905607  0.867522  0.048409
             Under     0.870673  0.767113  0.020820
             Over      0.999890  0.844299  0.091511
SVM          Original  0.921063  0.902687  0.082132
             Under     0.888454  0.696340  0.009238
             Over      0.900787  0.732279  0.022778

<h2> Final remarks

<p> Our conclusion was that the model for the support vector machine was the best model for our business problem. The main reason for this was due to fairly good overall scores on the train and test set, and it had the lowest PCL-score. PCL score is a parameter that measures the amount of times our model predicts true-Negatives. We decided that this was an important measurment seen from a business perspective and therefore the model based on the support vector machine algorithm and trained on the undersampled dataset had best results. For more informmation; read our thesis (its in the repository).